In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install additional packages to handle lemmatization.
!pip install symspellpy vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=187462 sha256=42d144433b963b9d64730a35f090352c1929d0acd726eff180553bbc1fc83847
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [3]:
# Change directory.
import os
os.chdir("/content/drive/MyDrive/ML Project 2")
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1RENc7fDlwfPskohAfMC6E1RzoMXRPI1K/ML Project 2'

## Import dependencies

In [4]:
from utility.paths import DataPath
from preprocessing import Preprocessing
from models.roberta import RoBERTa

import pandas as pd
from tqdm.auto import tqdm

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Create training and testing preprocessing object.
train_prep = Preprocessing([DataPath.TRAIN_NEG_FULL, DataPath.TRAIN_POS_FULL])
test_prep = Preprocessing([DataPath.TEST], is_test=True)

In [5]:
# Declare params for BERT.
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3

In [6]:
# Declare GRU model.
roberta = RoBERTa(weight_path="/content/drive/MyDrive/ML Project 2/weights/roberta",
                 submission_path="/content/drive/MyDrive/ML Project 2/submissions/roberta",
                 max_length=MAX_LEN,
                  is_weight=True)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/ML Project 2/weights/roberta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


## Preprocessing

In [ ]:
# Retrieve preprocessing steps declared in GRU class for both train and test data.
for step in tqdm(bert.preprocessing(), desc="Preprocessing train data"):
    getattr(train_prep, step)()

for step in tqdm(bert.preprocessing(is_train=False), desc="Preprocessing test data"):
    getattr(test_prep, step)()

Preprocessing train data:   0%|          | 0/7 [00:00<?, ?it/s]

Executing: `drop_duplicates`
Executing: `remove_tag`
Executing: `strip`
Executing: `remove_ellipsis`
Executing: `reconstruct_emoji`


100%|██████████| 2268591/2268591 [00:18<00:00, 120802.38it/s]


Executing: `remove_extra_space`


100%|██████████| 2268591/2268591 [00:02<00:00, 1108200.90it/s]


...

Executing: `remove_extra_space`


100%|██████████| 10000/10000 [00:00<00:00, 793834.51it/s]

Executing: `remove_space_around_emoji`


Executing: `remove_extra_space`


100%|██████████| 10000/10000 [00:00<00:00, 885341.21it/s]


In [ ]:
# Retrieve the preprocessed df.
train_data = train_prep.__get__()
test_data = test_prep.__get__()

In [ ]:
# Export the dataframes. For training frames, shuffles.
train_data = train_data.sample(frac=1)
train_data.to_csv(DataPath.BERT_TRAIN, index=False)

test_data.to_csv(DataPath.BERT_TEST, index=False)

## Training

In [ ]:
# Read the dataframe
train_df = pd.read_csv(DataPath.BERT_TRAIN)
train_df.head()

,text,label
0,seen while surfing : what a solution ! if you ...,1.0
1,something n40 would prevent ! smh i feel for d...,0.0
2,nah betch u owe me so i want food and i get to...,1.0
3,mine always get eaten before they get going . ...,0.0
4,i never got to go i'm 23 & still dream about i...,0.0


In [ ]:
# Drop nan in the df
train_df.dropna(inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268575 entries, 0 to 2268590
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   text    object 
 1   label   float64
dtypes: float64(1), object(1)
memory usage: 51.9+ MB


### Splitting data

In [ ]:
# Create X and y to feed into GRU
X, y = train_df['text'].values, train_df['label'].values

In [ ]:
# Start the training process
roberta.train(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS)

Tokenizing data:   0%|          | 0/2041717 [00:00<?, ?it/s]

Tokenizing data:   0%|          | 0/226858 [00:00<?, ?it/s]

Training steps: 191409
Model summary
Model: "tf_roberta_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 124647170 (475.49 MB)
Trainable params: 124647170 (475.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Fitting model
Epoch 1/3


Generating features:   0%|          | 0/2041717 [00:00<?, ?it/s]

  63804/Unknown - 13244s 207ms/step - loss: 0.3021 - accuracy: 0.8677

...

Generating features:   0%|          | 0/226858 [00:00<?, ?it/s]

63804/63804 [==============================] - 13729s 215ms/step - loss: 0.2500 - accuracy: 0.8945 - val_loss: 0.2631 - val_accuracy: 0.8926
Epoch 3/3


Generating features:   0%|          | 0/2041717 [00:00<?, ?it/s]

63804/63804 [==============================] - ETA: 0s - loss: 0.2176 - accuracy: 0.9097

Generating features:   0%|          | 0/226858 [00:00<?, ?it/s]

## Predicting

In [7]:
# Read preprocessed test data
test_df = pd.read_csv(DataPath.BERT_TEST)
test_df.head()

,ids,text
0,1,sea doo pro sea scooter :( sports with the por...
1,2,shucks well i work all week so now i can't com...
2,3,i cant stay away from bug thats my baby
3,4,no ma'am ! ! ! lol im perfectly fine and not c...
4,5,"whenever i fall asleep watching the tv , i alw..."


In [9]:
# Retrieve `text` column for predicting
X_test = test_df["text"]
roberta.predict(X_test)

Generating predictions:   0%|          | 0/10000 [00:00<?, ?it/s]

Saving predictions
